In [30]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,auc,roc_curve,f1_score
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE,ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
RandomUnderSampler
from sklearn  import preprocessing
import  xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D
from keras.layers import Conv1D, MaxPooling1D,Dropout
from keras.layers import Dense,Flatten
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
#import utils.tools as utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D, MaxPooling1D,Dropout,AveragePooling1D,GlobalAvgPool1D,GlobalMaxPool1D
from keras.models import Model
import tensorflow as tf
import lightgbm as lgb

In [45]:
#训练集
train_data=pd.read_csv("./train7.4.csv")
#测试集
test_data=pd.read_csv("./test7.4.csv")
#训练标签
train_label=pd.read_csv("./y_train.csv")
train_label=train_label.iloc[:,1]
print(train_label.shape)
#删除前两列无意义的特征
train_data=train_data.iloc[:,2:].values
train_label=train_label.values.ravel()
print(train_data.shape)
print(train_label.shape)
test_data=test_data.iloc[:,2:].values
print(test_data.shape)

(784000,)
(784000, 96)
(784000,)
(340043, 96)


In [4]:
#过采样
model_smote=SMOTE(k_neighbors=20)
x,y=model_smote.fit_resample(train_data,train_label)

In [46]:
#归一化
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
X=std.fit_transform(x)
X_ind=std.transform(test_data)
print(X_ind.shape)

(340043, 96)


In [47]:
train_data_inp=std.transform(train_data)
train_data_inp=train_data_inp.reshape(train_data_inp.shape[0],train_data_inp.shape[1],1)
train_data_inp=train_data_inp.astype('float32')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [7]:
#模型输入的张量格式转换
#X_all是所有训练集
X_all=X.reshape(X.shape[0],X.shape[1],1)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

X_ind=X_ind.reshape(X_ind.shape[0],X_ind.shape[1],1)
X_ind=X_ind.astype('float32')

X_all=X_all.astype('float32')
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_all=to_categorical(y,2)
y_train=to_categorical(y_train,2)
y_test=to_categorical(y_test,2)

In [8]:
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 5, padding = 'same', activation= 'relu'))
#model.add(MaxPooling1D(pool_size = 2))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(filters = 128, kernel_size =  3, padding = 'same', activation= 'relu'))
#model.add(MaxPooling1D(pool_size = 2))
model.add(AveragePooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation = 'relu',name="Dense_128"))
model.add(Dense(64, activation = 'relu',name="Dense_64"))
model.add(Dense(32, activation = 'relu',name="Dense_32"))
model.add(Dense(2, activation = 'softmax',name="Dense_2"))
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics =['accuracy'])#rmsprop

#15-0.936
early_stopping=EarlyStopping(monitor='loss',patience=20,verbose=1)
model.fit(X_all,y_all,batch_size=1000,epochs=30,callbacks=[early_stopping])

Epoch 1/30
1400/1400 [==============================] - 133s 95ms/step - loss: 0.1529 - accuracy: 0.9466
Epoch 2/30
1400/1400 [==============================] - 132s 95ms/step - loss: 0.0870 - accuracy: 0.9659
Epoch 3/30
1400/1400 [==============================] - 132s 94ms/step - loss: 0.0770 - accuracy: 0.9690
Epoch 4/30
1400/1400 [==============================] - 132s 94ms/step - loss: 0.0729 - accuracy: 0.9703
Epoch 5/30
1400/1400 [==============================] - 130s 93ms/step - loss: 0.0694 - accuracy: 0.9717s - loss: 0.0694 
Epoch 6/30
1400/1400 [==============================] - 131s 93ms/step - loss: 0.0675 - accuracy: 0.9722
Epoch 7/30
1400/1400 [==============================] - 132s 94ms/step - loss: 0.0657 - accuracy: 0.9729
Epoch 8/30
1400/1400 [==============================] - 129s 92ms/step - loss: 0.0649 - accuracy: 0.9730
Epoch 9/30
1400/1400 [==============================] - 130s 93ms/step - loss: 0.0633 - accuracy: 0.9735s - loss: 0.0633 - accuracy
Epoch 10/30

In [ ]:
ind_pri_7=model.predict_classes(X_ind)

In [29]:
ind_pri_7=pd.DataFrame(ind_pri_7)
ind_pri_7.to_csv("./cnn_test_label.csv",index=0,header=None)

In [55]:
train_pri=model.predict_proba(train_data_inp)

C:\Users\Lenovo\myconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [56]:
train_pri=pd.DataFrame(train_pri)
train_pri.to_csv("./cnn_train_pro.csv",index=0,header=None)